# Initialize libraries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
dir_home = str(Path.home())
if not any([Path(_p).name=='ILECTools' for _p in sys.path]):
    sys.path.append(dir_home + '/dev/ILECTools') # is where it is on this machine
    
    
def to_multi_columns(self):
    self.columns = pd.MultiIndex.from_tuples([tuple(c.split('_')) for c in self.columns])
    return self
pd.DataFrame.to_multi_columns = to_multi_columns

Working notes
* Site: https://www.soa.org/resources/experience-studies/2017/2009-13-indiv-life-ins-mort-exp/
* Appendices: https://www.soa.org/globalassets/assets/files/research/exp-study/2009-13-indiv-life-ins-mort-exp-appendix.xlsx

Note: the zip of the csv is also at the site.

This work is based off duplicating appendix G.  My old data selection did not quite duplicate appendix G.  The totals in the published appendix across products do not match the rows, so I'm not going to sweat duplicating that here.

# Read data, add columns

In [2]:
%%time
data = pd.read_parquet(dir_home + '/data/ilec/Data/Processed/2021_published/ILEC 2009-18 20210528_v1.parquet')

data_g = data[  (data.issue_age >17)
           & (data.observation_year < 2014)
           & (data.duration<26)
           & (data.soa_post_level_term_indicator != 'Post Level Term')
          ].copy()
del data

CPU times: user 28.4 s, sys: 25.2 s, total: 53.7 s
Wall time: 46 s


Add other columns

## Add other columns

In [30]:
data_g['dur_band1'] = 'n/a' # a default, which will get overridden
for band, durs in {
    '01':[1],
    '02':[2],
    '03':[3],
    '04-05':[4,5],
    '06-10':range(6,11),
    '11-15':range(11,16),
    '16-20':range(16,21),
    '21-25':range(21,26)}.items():
    data_g.loc[data_g.duration.isin(durs), 'dur_band1'] = band
    
data_g['ia_band1'] = '18-24'
for a in range(25, 95, 5):
    data_g.loc[data_g.issue_age.between(a, a+4), 'ia_band1'] = f'{a}-{a+4}'
data_g.loc[data_g.issue_age>94, 'ia_band1'] = '95+'

data_g['iy_band1'] = '1900-1989'
data_g.loc[data_g.issue_year.between(1990, 1999), 'iy_band1'] = '1990-1999'
data_g.loc[data_g.issue_year.between(2000, 2009), 'iy_band1'] = '2000-2009'
data_g.loc[data_g.issue_year >=  2010, 'iy_band1'] = '2010-'


# I don't like 'Not Level Term' and "N/A (Not Term)" both being in there, should probably have consolidated them
data_g['ltp'] = (data_g.soa_anticipated_level_term_period
                 .replace({'20 yr anticipated': '20 yr or N/A (Not Term)',
                            'N/A (Not Term)': '20 yr or N/A (Not Term)'}).str.replace(' anticipated', '')
                )


array(['20 yr or N/A (Not Term)', 'Unknown', 'Not Level Term', '30 yr',
       '10 yr', '15 yr', ' 5 yr', '25 yr'], dtype=object)


       offsetcol='expected_death_qx2015vbt_by_policy'
      , formula = """number_of_deaths ~
    + C(class_key)
    + C(dur_band1, Treatment(reference='06-10'))
    + C(face_amount_band, Treatment(reference='  100000-249999'))
    + C(ia_band1, Treatment(reference='40-44'))
    + C(gender)
    + C(observation_year)
    + C(insurance_plan, Treatment(reference='Term'))
    + C(ltp, Treatment(reference='20 yr or N/A (Not Term)'))
    + C(iy_band1)

In [72]:
for f in ['face_amount_band', 'gender', 'insurance_plan']:
    print(f, set(data_g[f].unique()).symmetric_difference(data_app[f].unique()))

face_amount_band set()
gender set()
insurance_plan {'ET-RPU'}


Quite close to report - not exact, different dataset, but IMO not materially different

In [5]:

tmp = data_g.pivot_table(index= 'face_amount_band',
                        columns='insurance_plan',
                        values=['policy_actual', 'amount_actual', 'amount_2015vbt', 'policy_2015vbt'],
                        aggfunc=np.sum,
                        margins=True)


pd.concat([(tmp[f'{m}_actual'] / tmp[f'{m}_2015vbt']) for m in ['policy', 'amount']]
, keys=['policy', 'amount'],
axis=1).style.format('{:,.1%}')

# Read from the old pickle: matches by line, total in downloaded spreadsheet is wrong itself

In [149]:
%%time
data_app = pd.read_pickle('/home/brian/data/ilec/Studies/Development/201706_Dataset/data/soa.det201706_sum3_extra.pkl')
# There are some errors: no exposure or no qx, which have to be excluded to run models with log offset
data_app = data_app[data_app['expected_death_qx2015vbt_by_policy']>0.]

data_app['structure'] = data_app.structure.str.replace('NonSmoker', 'Nonsmoker')
data_app['face_band3'] = data_app['face_amount_band'].map(lambda b: {' 2500000-4999999':' 2500000+'
                                                                     ,' 5000000-9999999':' 2500000+'
                                                                     ,'10000000+':' 2500000+'}.get(b, b))
data_app['class_key'] = data_app.apply(lambda s: '{} {}'.format(s.structure, s.preferred_class), axis=1)


CPU times: user 38.5 s, sys: 787 ms, total: 39.3 s
Wall time: 39.2 s


In [229]:
# I don't like 'Not Level Term' and "N/A (Not Term)" both being in there, should probably have consolidated them
data_app['ltp'] = (data_app.soa_anticipated_level_term_period
                 .replace({'20 yr anticipated': '20 yr or N/A (Not Term)',
                            'N/A (Not Term)': '20 yr or N/A (Not Term)'}).str.replace(' anticipated', '')
                )

In [231]:
from functools import reduce
import operator

def getSubset(data_app, cri, **kwargs):
    """Apply the criteria functions to each column and return a subset of the given dataset.
    The criteria are a list of (seriesname, function) pairs.  The function returns True or False and is mapped on the series.
    """
    rowcri = kwargs.pop('rowcri', []) #criteria to be applied to each row; more complicated, functions of the row
    return data_app[reduce(operator.and_
                            , map(lambda kf: data_app[kf[0]].map(kf[1]), cri)
                   #  + map(lambda f: data_app.apply(lambda s: f(s), axis=1) #apply each function f in rowcri to each row
                   #        , rowcri)
                          )
                   ]

cri = {}
# Appendix G - updated for final
cri.update({'G':  [('insurance_plan', lambda a: a not in ['ET-RPU'])  #  , 'Other' is not excluded in the report.
    ,('soa_post_level_term_indicator', lambda a: a != 'Post Level Term')
    ,('source_name', lambda sn: sn in ['LSS 09-13','ILEC LSS 09'])
    ,('dur_band1', lambda d: d != '26-150') #select only
    ,('ia_band1', lambda a: a not in [' 0', ' 1-4', ' 5-9', '10-17']) #, '95+'  
    #,('face_min', lambda fb: 100000. <= fb) # is difference from 2008-9 study
    ]})
data_g_report = getSubset(data_app, cri['G'])


# rename value columns I need to compare with new pull more easily
data_g_report = data_g_report.rename(columns={'expected_death_qx2015vbt_by_amount': 'amount_2015vbt', 
                              'expected_death_qx2015vbt_by_policy': 'policy_2015vbt',
                               'number_of_deaths':'policy_actual',
                                'death_claim_amount':'amount_actual'})


data_g_report = data_g_report.rename(columns={'class_key':'uw'})
data_g_report = data_g_report.replace({'uw':{'Nonsmoker 4 2': 'N/4/2',
 'Nonsmoker nan': 'N/1/1',
 'Nonsmoker 3 1': 'N/3/1',
 'Smoker 2 2': 'S/2/2',
 'Nonsmoker 3 2': 'N/3/2',
 'Unknown nan': 'U/1/1',
 'Nonsmoker 2 2': 'N/2/2',
 'Nonsmoker 2 1': 'N/2/1',
 'Nonsmoker 3 3': 'N/3/3',
 'Smoker 2 1': 'S/2/1',
 'Nonsmoker 4 4': 'N/4/4',
 'Nonsmoker 4 1': 'N/4/1',
 'Nonsmoker 4 3': 'N/4/3',
 'Smoker nan': 'S/1/1'}})


pt = data_g_report.pivot_table(index='face_amount_band', 
                        columns='insurance_plan',
                        values=['policy_actual', 'amount_actual'], aggfunc=np.sum,
                        margins=True)
pd.options.display.float_format = '{:,.0f}'.format
pt*1.

amount_actual                                               \
insurance_plan           Other           Perm           Term             UL   
face_amount_band                                                              
    1-9999             149,770    490,099,587     10,201,613     25,488,224   
   10000-24999         626,052  1,253,170,039     58,298,633    342,267,677   
   25000-49999         513,923  1,346,558,334    168,715,602  1,208,921,867   
   50000-99999       1,062,554  1,677,912,993    733,785,529  2,493,508,273   
  100000-249999      6,056,902  2,857,113,261  5,454,633,611  4,074,937,195   
  250000-499999      2,950,363  1,756,070,757  6,448,818,487  2,020,023,613   
  500000-999999      3,064,061  1,615,584,744  6,516,708,555  1,798,881,702   
 1000000-2499999     8,200,000  1,853,678,981  7,267,674,372  2,821,912,574   
 2500000-4999999             0    549,373,069  1,514,281,938  1,410,989,594   
 5000000-9999999     9,000,000    267,292,703  1,006,284,307  2,060,573,519   
10000000+           12,500,000    268,364,314    635,000,000  1,914,282,498   
All                 44,123,625 13,935,218,782 29,814,402,647 20,171,786,736   

                                                                            \
insurance_plan             ULSG            VL          VLSG            All   
face_amount_band                                                             
    1-9999              305,277     1,323,618     4,654,345    532,222,434   
   10000-24999       11,259,774    10,852,755    12,501,255  1,688,976,185   
   25000-49999       69,228,109    74,897,568    26,232,215  2,895,067,618   
   50000-99999      280,330,331   788,206,976    52,460,409  6,027,267,065   
  100000-249999   1,057,453,506 1,305,717,285   282,070,578 15,037,982,338   
  250000-499999     986,192,151   900,425,282   272,509,619 12,386,990,272   
  500000-999999   1,254,444,705   841,305,119   268,255,626 12,298,244,512   
 1000000-2499999  2,391,223,507 1,012,210,933   370,807,796 15,725,708,163   
 2500000-4999999  1,536,540,770   330,426,780   140,933,340  5,482,545,491   
 5000000-9999999  1,965,115,624   309,641,208   112,951,079  5,730,858,440   
10000000+         1,584,978,788   252,028,532    75,159,873  4,742,314,005   
All              11,137,072,542 5,827,036,056 1,618,536,135 82,548,176,523   

                 policy_actual                                              \
insurance_plan           Other    Perm    Term      UL   ULSG     VL  VLSG   
face_amount_band                                                             
    1-9999                   7 105,162   1,961   4,042     39    224   858   
   10000-24999              24  92,002   4,129  23,322    823    638   950   
   25000-49999              16  42,812   6,067  39,570  2,431  2,285   959   
   50000-99999              18  26,966  13,603  41,903  4,908 13,417   875   
  100000-249999             36  21,157  43,781  31,880  8,072 10,003 2,142   
  250000-499999             10   5,503  22,875   6,595  3,141  2,943   909   
  500000-999999              6   2,591  11,905   3,073  2,088  1,426   451   
 1000000-2499999             7   1,416   6,028   2,183  1,795    779   278   
 2500000-4999999             0     167     495     444    468    104    45   
 5000000-9999999             1      43     183     367    349     50    19   
10000000+                    1      22      45     157    125     17     5   
All                        126 297,841 111,072 153,536 24,239 31,886 7,491   

                          
insurance_plan       All  
face_amount_band          
    1-9999       112,293  
   10000-24999   121,888  
   25000-49999    94,140  
   50000-99999   101,690  
  100000-249999  117,071  
  250000-499999   41,976  
  500000-999999   21,540  
 1000000-2499999  12,486  
 2500000-4999999   1,723  
 5000000-9999999   1,012  
10000000+            372  
All              626,191

# Fit the models

Results are not good.   Male mortality < female, other problems?

## Summarize from new data version and fit models

In [255]:
valcols = ['policy_actual', 'policy_2015vbt', 'amount_actual', 'amount_2015vbt']

data_g_mod = data_g.pivot_table(index=['uw', 'dur_band1', 'face_amount_band', 'ia_band1', 'gender', 'observation_year', 'insurance_plan', 'ltp', 'iy_band1'],
                                values=valcols,
                                aggfunc=np.sum,
                                fill_value=0 )
data_g_mod = data_g_mod[data_g_mod['amount_2015vbt']>0].reset_index() # to remove the few records with no tabular values, which cannot be handled


In [268]:
%%time
# Fit the models.  This all takes about 5 min.

from importlib import reload
from ilectools import glmtools
glmtools = reload(glmtools)
from ilectools.glmtools import glmrun

mod = {}

import statsmodels.api as sm, statsmodels.formula.api as smf

mod = {f'{m}model':glmrun(
        offsetcol=f'{m}_2015vbt'
      , formula = f"""{m}_actual ~
    + C(uw, Treatment(reference='N/2/1'))
    + C(dur_band1, Treatment(reference='06-10'))
    + C(face_amount_band, Treatment(reference='  100000-249999'))
    + C(ia_band1, Treatment(reference='40-44'))
    + C(gender)
    + C(observation_year)
    + C(insurance_plan, Treatment(reference='Term'))
    + C(ltp, Treatment(reference='20 yr or N/A (Not Term)'))
    + C(iy_band1)
    """
      , data = data_g_mod
          )
       for m in ['policy', 'amount']
      }


2022-04-26 18:47:08.876476 Fitting ...
2022-04-26 18:47:48.913634 ... fit.
2022-04-26 18:47:48.913786 Fitting ...
2022-04-26 18:48:36.613038 ... fit.
CPU times: user 6min 41s, sys: 2min 10s, total: 8min 52s
Wall time: 1min 27s


In [269]:
%%time
# Add both models predctions to the dataframe data_g.
from ilectools.glmtools import addPred
for _k in mod.keys():
    # Keep the names in the model for convenience: writing exhibits and the like.    
    mod[_k].name = _k
    # Add the predictions to the detailed dataframe to look within each cell: by amount and by count
    # Pairs of new (predicted) column name and corresponding offset
    addPred(mod[_k]
                , [('{}_{}'.format(_m, _k)
                    ,'{}_2015vbt'.format(_m))
                   for _m in ['policy','amount']]
                ,  data = data_g_mod
       )

2022-04-26 18:48:47.663751 Predicting with offset policy_2015vbt
2022-04-26 18:48:47.691583 Predicting with offset amount_2015vbt
2022-04-26 18:48:47.728498 ... predicted.
2022-04-26 18:48:57.776828 Predicting with offset policy_2015vbt
2022-04-26 18:48:57.798439 Predicting with offset amount_2015vbt
2022-04-26 18:48:57.824002 ... predicted.
CPU times: user 21.5 s, sys: 2.53 s, total: 24 s
Wall time: 21.2 s


## Fit from old data

In [267]:

mod_report = {f'{m}model':glmrun(
        offsetcol=f'{m}_2015vbt'
      , formula = f"""{m}_actual ~
    + C(uw, Treatment(reference='N/2/1'))
    + C(dur_band1, Treatment(reference='06-10'))
    + C(face_amount_band, Treatment(reference='  100000-249999'))
    + C(ia_band1, Treatment(reference='40-44'))
    + C(gender)
    + C(observation_year)
    + C(insurance_plan, Treatment(reference='Term'))
    + C(ltp, Treatment(reference='20 yr or N/A (Not Term)'))
    + C(iy_band1)
    """
      , data = data_g_report
          )
       for m in ['policy', 'amount']
      }


2022-04-26 18:44:24.657963 Fitting ...
2022-04-26 18:45:33.393045 ... fit.
2022-04-26 18:45:33.393498 Fitting ...
2022-04-26 18:47:08.829896 ... fit.


## Compare model factors old and new

In [270]:
comp = pd.concat([pd.DataFrame({k:mod[k].prettyParams() for k in mod.keys()}),
 pd.DataFrame({k:mod_report[k].prettyParams() for k in mod_report.keys()})],
keys=['new', 'old'],
axis=1)
comp = pd.concat([comp, pd.concat([comp['new']-comp['old']], axis=1, keys=['diff'])], axis=1)
comp = comp.T.swaplevel(0, 1).sort_index().T
comp.style.format({c:'{:,.1%}' if c[1]=='diff' else '{:,.4f}' for c in comp.columns})

In [272]:
m = mod['amountmodel']

In [273]:
m.compareFactorsAE()

NameError: name 'getSubgraphs' is not defined